In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy
from torch.utils.data import Dataset, DataLoader
import glob
from PIL import Image


class datasetloader(Dataset):
    def __init__(self, path, transform=None):
        self.classes   = os.listdir(path)
        self.classes = [i for i in self.classes if not i.startswith('.')]
        self.path      = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform
        
        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
        self.file_list = files
        files = None
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
        return im, classCategory

In [4]:
class CNN_1(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(CNN_1, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=n_feature, kernel_size=5)
        self.conv2 = nn.Conv2d(n_feature, n_feature, kernel_size=5)
        self.fc1 = nn.Linear(n_feature*53*53, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x, verbose=False):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(-1, self.n_feature*53*53)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x



In [5]:
#function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

input_size  = 224*224*3   # images are 224*224 pixels and has 3 channels because of RGB color
output_size = 2      # there are 2 classes---Cat and dog

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 64


# define training and test data directories
data_dir = './dataset/'
train_dir = os.path.join(data_dir, 'training_set/')
test_dir = os.path.join(data_dir, 'test_set/')


#create transformers
image_size = (224, 224)
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
                                transforms.Resize(image_size), 
                                                    transforms.ToTensor(), 
                                transforms.Normalize(mean, std)])
test_transforms = transforms.Compose([
                                transforms.Resize(image_size), 
                                transforms.ToTensor(), 
                                transforms.Normalize(mean, std)])


 ## read data set using the custom class
train_dataset = datasetloader(train_dir, transform=train_transform)
test_dataset = datasetloader(test_dir, transform=test_transforms)

## load data using utils
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
     num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
     num_workers=num_workers)

accuracy_list = []

def train(epoch, model):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        #print(data[0].shape)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10  >5:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss                                                               
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

# Training settings  for model 1
n_features = 16 # hyperparameter

model_cnn1 = CNN_1(input_size, n_features, output_size)
optimizer = optim.SGD(model_cnn1.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model_cnn1)))

for epoch in range(0, 5):
    train(epoch, model_cnn1)
    test(model_cnn1)


    

Number of parameters: 2255392
Train Epoch: 0 [384/8000 (5%)]	Loss: 1.394188
Train Epoch: 0 [448/8000 (6%)]	Loss: 1.855660
Train Epoch: 0 [512/8000 (6%)]	Loss: 1.621260
Train Epoch: 0 [576/8000 (7%)]	Loss: 1.382897
Train Epoch: 0 [1024/8000 (13%)]	Loss: 0.768415
Train Epoch: 0 [1088/8000 (14%)]	Loss: 0.703205
Train Epoch: 0 [1152/8000 (14%)]	Loss: 0.737521
Train Epoch: 0 [1216/8000 (15%)]	Loss: 0.675895
Train Epoch: 0 [1664/8000 (21%)]	Loss: 0.720308
Train Epoch: 0 [1728/8000 (22%)]	Loss: 0.696979
Train Epoch: 0 [1792/8000 (22%)]	Loss: 0.729697
Train Epoch: 0 [1856/8000 (23%)]	Loss: 0.702791
Train Epoch: 0 [2304/8000 (29%)]	Loss: 0.731507
Train Epoch: 0 [2368/8000 (30%)]	Loss: 0.816904
Train Epoch: 0 [2432/8000 (30%)]	Loss: 0.695459
Train Epoch: 0 [2496/8000 (31%)]	Loss: 0.692677
Train Epoch: 0 [2944/8000 (37%)]	Loss: 0.671866
Train Epoch: 0 [3008/8000 (38%)]	Loss: 0.682580
Train Epoch: 0 [3072/8000 (38%)]	Loss: 0.659770
Train Epoch: 0 [3136/8000 (39%)]	Loss: 0.696951
Train Epoch: 0 [35